In [51]:
import DataRetriever as dr
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

In [52]:
retriever = dr.DataRetriever()
year_2 = retriever.get_data("All-Subsystems-hour-Year2.pkl")
year_2["Timestamp"] = pd.to_datetime(year_2["Timestamp"])

In [53]:
# Remove the "-5" part from the timestamp, since we do not need to convert between timezones, so we see the time as local
for row in year_2.index:
    dt = year_2["Timestamp"][row]
    year_2.at[row, "Timestamp"] = pd.Timestamp(year=dt.year, month=dt.month, day=dt.day,
                                           hour=dt.hour, minute=dt.minute, second=dt.second)

# Cast the Timestamp column to be of type DateTime
year_2["Timestamp"] = pd.to_datetime(year_2["Timestamp"])

In [54]:
# Set the index to be the timestamp to keep it when finding frequent items
year_2.index = year_2["Timestamp"]

In [55]:
pv_columns = [column for column in year_2.columns if column.startswith("PV_Watts3PhTotalW3PhT1")]
pv_data = year_2[pv_columns]

In [56]:
pv_data[pv_data < 10] = 0

C:\Users\madsc\AppData\Local\Temp\ipykernel_19964\3308511092.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\madsc\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:3718: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [57]:
# Create a DataFrame containing One-Hot-Encoded hour of each TimeStamp in the data
pv_data["Hour"] = pv_data.index.hour
dummy_hours = pd.get_dummies(pv_data["Hour"])  # One-hot encoding

C:\Users\madsc\AppData\Local\Temp\ipykernel_19964\4064856359.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [58]:
pv_data = pv_data.merge(right=dummy_hours, how='left', left_index=True, right_index=True)
del pv_data["Hour"]

In [59]:
pv_data_bool = pv_data.astype(bool)

In [60]:
pv_data_bool

,PV_Watts3PhTotalW3PhT1,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-02-01 00:00:00,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2015-02-01 01:00:00,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2015-02-01 02:00:00,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2015-02-01 03:00:00,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2015-02-01 04:00:00,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-31 19:00:00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2016-01-31 20:00:00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2016-01-31 21:00:00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [61]:
frequent_itemsets = fpgrowth(pv_data_bool, min_support=0.01, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.041657,(0)
1,0.042000,(1)
2,0.041543,(2)
3,0.041657,(3)
4,0.041657,(4)
5,0.041657,(5)
6,0.041657,(6)
7,0.503196,(PV_Watts3PhTotalW3PhT1)
8,0.041657,(7)
9,0.041657,(8)


In [62]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

In [63]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules = rules[(rules['antecedent_len'] == 1)]

In [64]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(7),(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.035380,0.849315,1.687843,0.014418,3.296975,1
1,(8),(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.039945,0.958904,1.905629,0.018984,12.088907,1
2,(9),(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040516,0.972603,1.932852,0.019554,18.133360,1
3,(10),(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040630,0.975342,1.938297,0.019668,20.148178,1
4,(11),(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040744,0.978082,1.943741,0.019782,22.666700,1
5,(12),(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040630,0.975342,1.938297,0.019668,20.148178,1
6,(13),(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040516,0.972603,1.932852,0.019554,18.133360,1
7,(14),(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040516,0.972603,1.932852,0.019554,18.133360,1
8,(15),(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040288,0.967123,1.921963,0.019326,15.111133,1
9,(16),(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.039146,0.939726,1.867516,0.018185,8.242436,1


In [65]:
for index, row in rules.iterrows():
    rules.at[index, "antecedents"] = list(rules.at[index, "antecedents"])[0]

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,7,(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.035380,0.849315,1.687843,0.014418,3.296975,1
1,8,(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.039945,0.958904,1.905629,0.018984,12.088907,1
2,9,(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040516,0.972603,1.932852,0.019554,18.133360,1
3,10,(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040630,0.975342,1.938297,0.019668,20.148178,1
4,11,(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040744,0.978082,1.943741,0.019782,22.666700,1
5,12,(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040630,0.975342,1.938297,0.019668,20.148178,1
6,13,(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040516,0.972603,1.932852,0.019554,18.133360,1
7,14,(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040516,0.972603,1.932852,0.019554,18.133360,1
8,15,(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.040288,0.967123,1.921963,0.019326,15.111133,1
9,16,(PV_Watts3PhTotalW3PhT1),0.041657,0.503196,0.039146,0.939726,1.867516,0.018185,8.242436,1


In [67]:
import plotly.express as px
fig = px.bar(data_frame=rules, x="antecedents", y="support")
fig.show()